In [ ]:
# Install dependencies
%pip install -e '.[kafka-python]'

# Import the reliable topic creation functions
from diaspora_event_sdk.sdk.kafka_client import (
    reliable_client_creation,
    KafkaProducer,
    KafkaConsumer,
)
from diaspora_event_sdk import Client as GlobusClient
import json
import uuid
import time
from datetime import datetime

# Initialize client
# os.environ["DIASPORA_SDK_ENVIRONMENT"] = "local"
c = GlobusClient()

In [ ]:
# Run reliable_client_creation to set up client and test basic flow
reliable_client_creation()

In [ ]:
# Function to randomly create a topic, produce, and consume messages
def random_topic_produce_consume():
    """Create a random topic, produce 3 messages, and consume them."""
    topic_name = f"topic-{str(uuid.uuid4())[:5]}"
    kafka_topic = f"{c.namespace}.{topic_name}"

    print(f"\n{'=' * 60}")
    print(f"Creating topic: {topic_name}")
    print(f"Kafka topic: {kafka_topic}")
    print(f"{'=' * 60}")

    # Create topic
    topic_result = c.create_topic(topic_name)
    print(f"Topic creation: {json.dumps(topic_result, indent=2, default=str)}")

    if topic_result.get("status") != "success":
        print(f"Failed to create topic: {topic_result}")
        return None

    time.sleep(3)  # Wait for topic to be ready

    # Produce messages
    print(f"\nProducing 3 messages to {kafka_topic}...")
    producer = KafkaProducer(kafka_topic)
    for i in range(3):
        message = {
            "message_id": i + 1,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "content": f"Message {i + 1} from random_topic_produce_consume",
        }
        future = producer.send(kafka_topic, message)
        result = future.get(timeout=30)
        print(f"  Produced message {i + 1}: offset={result.offset}")
    producer.close()

    time.sleep(2)  # Wait before consuming

    # Consume messages
    print(f"\nConsuming messages from {kafka_topic}...")
    consumer = KafkaConsumer(kafka_topic, auto_offset_reset="earliest")
    messages = consumer.poll(timeout_ms=10000)
    consumed_count = 0
    for tp, msgs in messages.items():
        for message in msgs:
            data = json.loads(message.value.decode("utf-8"))
            consumed_count += 1
            print(f"  Consumed message {consumed_count}: {data}")
    consumer.close()
    print(f"\nTotal messages consumed: {consumed_count}")

    # Delete topic
    print(f"\nDeleting topic {topic_name}...")
    delete_result = c.delete_topic(topic_name)
    print(f"Topic deletion: {json.dumps(delete_result, indent=2, default=str)}")

    return kafka_topic


# Call the function three times
for i in range(5):
    print(f"\n\n{'#' * 60}")
    print(f"# Iteration {i + 1} of 5")
    print(f"{'#' * 60}")
    result = random_topic_produce_consume()
    if result:
        print(f"✓ Successfully completed iteration {i + 1}: {result}")
    else:
        print(f"✗ Failed iteration {i + 1}")
    if i < 2:  # Don't sleep after last iteration
        time.sleep(2)

In [ ]:
# Cleanup: Delete user at the end
print(f"\n{'=' * 60}")
print("Cleaning up: Deleting user...")
print(f"{'=' * 60}")
delete_user_result = c.delete_user()
print(f"User deletion: {json.dumps(delete_user_result, indent=2, default=str)}")
if delete_user_result.get("status") == "success":
    print("✓ User successfully deleted")
else:
    print(f"✗ Failed to delete user: {delete_user_result}")